<a href='http://www.holoviews.org'><img src="../../assets/pn_hv_gv_bk_ds_pa.png" alt="HoloViz logos" width="40%;" align="left"/></a>
<div style="float:right;"><h2>Exercises 3: Building Pipelines</h2></div>

### Exercise 3

In this exercise we will explore hvplot some more which we will build on in the next exercise to create linked visualization.

#### Loading the data 

We will be building a new visualization based on the same data we have cleaned and filtered in the rest of the tutorial. First we load the earthquakes `DataFrame` and filter to those with `>=7` magnitude:

In [ ]:
import numpy as np
import pandas as pd 
import xarray as xr
import panel as pn

import hvplot.pandas # noqa: adds hvplot method to pandas objects
import hvplot.xarray # noqa: adds hvplot method to xarray objects
pn.pane.DataFrame.max_rows = 1000

In [ ]:
df = pd.read_parquet('../../data/earthquakes-projected.parq')
df.time = df.time.astype('datetime64[ns]')
columns = ['time', 'mag', 'depth', 'latitude', 'longitude', 'place', 'type']
df = df.set_index(df.time)[columns]
most_severe = df[df.mag >= 7]

### Initial inspection of the depth data

Declare and display a depth float slider with the handle `depth_slider` (and named 'Minimum depth') that ranges between zero and 700 meters and verify that the depth values in `most_severe` lie in this range. Set the default value to the middle of this range.


<br><details><summary>Hint</summary><br>

You can use the `min()` and `max()` method on the `depth` `Series` of `most_severe` to check the range. To declare the slider, use a `pn.widgets.FloatSlider`.
</details>

In [ ]:
depth_slider = ... 
depth_slider

<details><summary>Solution</summary><br>

```python
depth_slider = pn.widgets.FloatSlider(name='Minimum depth', start=0, end=700, value=350)
depth_slider
```
    
```python
>> most_severe.depth.min()
4.2
>> most_severe.depth.max()
675.4
    
```

<br></details>

#### Exploring an interactive `DataFrame`

Now we will create a new interactive `DataFrame` called `dfi` with `sizing_mode='stretch_width'`.


<br><details><summary>Hint</summary><br>

Use the `.interactive` method on `most_severe` to create the interactive `DataFrame` called `dfi`
</details>



In [ ]:
dfi = ... # Interactive DataFrame version of most_severe

<details><summary>Solution</summary><br>

```python
dfi = most_severe.interactive(sizing_mode='stretch_width')
```

Now use this interactive `Dataframe` to filter the earthquakes deeper than specified by the `depth_slider`. Call this filtered dataframe `depth_filtered` and to view it conveniently, use the `.head()` method to see the first few entries.


<br><details><summary>Hint</summary><br>

Use the the regular pandas idiom to filter a `DataFrame` with `df[mask]` where `mask` is boolean mask. The only difference is instead of picking a fixed depth value to filter by, you can use the `depth_slider` widget instead.
</details>

In [ ]:
depth_filtered = ...
# Now display the head of this interactive dataframe

<details><summary>Solution</summary><br>

```python
depth_slider = pn.widgets.FloatSlider(name='Minimum depth', start=0, end=700, value=350)

dfi = most_severe.interactive(sizing_mode='stretch_width')
depth_filtered = dfi[dfi['depth'] < depth_slider]

depth_filtered.head()
```

#### Plotting the depth filtered data

For an initial plot, try calling `.hvplot()` and seeing what happens.


In [ ]:
# depth_filtered.hvplot()

Now let us focus on plotting the magnitude of the filtered earthquakes as a scatter plot with red cross (`x`) markers.

<br><details><summary>Hint</summary><br>

The magnitude column is called `mag`, you can set cross markers with `marker='x'` and to get a scatter plot you can use `kind='scatter'`.
</details>


In [ ]:
# Scatter plot of magnitude, filtered by depth with red cross markers

<details><summary>Solution</summary><br>

```python
depth_slider = pn.widgets.FloatSlider(name='Minimum depth', start=0, end=700, value=350)

dfi = most_severe.interactive(sizing_mode='stretch_width')
depth_filtered = dfi[dfi['depth'] < depth_slider]
depth_filtered.hvplot(y='mag', kind='scatter', color='red', marker='x')
```

In [ ]:
depth_slider = pn.widgets.FloatSlider(name='Minimum depth', start=0, end=700, value=350)

dfi = most_severe.interactive(sizing_mode='stretch_width')
depth_filtered = dfi[dfi['depth'] < depth_slider]
depth_filtered.hvplot(y='mag', kind='scatter', color='red', marker='x')